<a href="https://colab.research.google.com/github/eluisluzquadros/covid_impact_se/blob/main/step01_1_collection_grey_GoogleDrive_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [267]:

#!apt-get update # to update ubuntu to correctly run apt install


# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3




Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
cp: missing destination file operand after '/usr/lib/chromium-browser/chromedriver/usr/bin'
Try 'cp --help' for more information.
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backpor

### Import das dependencias

In [268]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import numpy as np
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [269]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [270]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impactSE/data/01_grey/2021/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2021-01-01.csv	2021-01-23.csv	2021-02-14.csv	2021-03-08.csv	2021-03-30.csv
2021-01-02.csv	2021-01-24.csv	2021-02-15.csv	2021-03-09.csv	2021-03-31.csv
2021-01-03.csv	2021-01-25.csv	2021-02-16.csv	2021-03-10.csv	2021-04-01.csv
2021-01-04.csv	2021-01-26.csv	2021-02-17.csv	2021-03-11.csv	2021-04-02.csv
2021-01-05.csv	2021-01-27.csv	2021-02-18.csv	2021-03-12.csv	2021-04-03.csv
2021-01-06.csv	2021-01-28.csv	2021-02-19.csv	2021-03-13.csv	2021-04-04.csv
2021-01-07.csv	2021-01-29.csv	2021-02-20.csv	2021-03-14.csv	2021-04-05.csv
2021-01-08.csv	2021-01-30.csv	2021-02-21.csv	2021-03-15.csv	2021-04-06.csv
2021-01-09.csv	2021-01-31.csv	2021-02-22.csv	2021-03-16.csv	2021-04-07.csv
2021-01-10.csv	2021-02-01.csv	2021-02-23.csv	2021-03-17.csv	2021-04-08.csv
2021-01-11.csv	2021-02-02.csv	2021-02-24.csv	2021-03-18.csv	2021-04-09.csv
2021-01-12.csv	2021-02-03.csv	2021-02-25.csv	2021-03-19.csv	2021-04-10.csv
2021-01-13.csv	2021-02-04.csv	2021-02-26.csv	2021-03-20.csv	2021-04-11.csv
2021-01-14.csv	2021-02-05

### Import chaves de api

In [271]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impactSE/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [272]:
GSE_RESULTS_CLASS = 'tF2Cxc' #'ZINbbc xpd O9g5cc uUPGi'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

'''
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  d_before = (date + DateOffset(days = 2)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = " + "after:" + date.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', '').split('&sa')[0],
         
    }
    
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))
'''

def search(query, date):

  # set chrome_options for chrome driver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  # define url de busca
  d = date.strftime('%Y%m%d')
  d_before = (date + DateOffset(days = 2)).date()
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, date.strftime('%Y-%m-%d'), d_before.strftime('%Y-%m-%d'), GSE_MAX_RESULTS)
  print(url)
  print("horizon = " + "after:" + date.strftime('%Y-%m-%d') +" and "+"before:"+d_before.strftime('%Y-%m-%d'))
  #print(query)
  print('\n')

  # get url com driver
  driver.get(url)
  # passa o html para o BeautifulSoup
  soup = BeautifulSoup(driver.page_source,'html.parser') 
  # busca a tag principal das news do google earth engine
  items = soup.find_all('div',{'class':GSE_RESULTS_CLASS})

  # copia o conteudo da class 'tF2Cxc' para uma lista
  result = []
  print('counts : ' + str(len(items)))

  for each in items:
    try:
      header = each.find('div',{'class':'yuRUbf'}).find('h3').text
      summary = each.find('div',{'class':'IsZvec'}).find('span').text
      link = each.find('a', href=True)['href']

      
      result.append({'date' : date.strftime('%Y-%m-%d'),
                     'header' : header,
                     'summary': summary,
                     'link' : link})
    except:
      pass

  # cria um csv com a lista  
  df = pd.DataFrame(result)
  return df

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [273]:
# Scraper 1 (2020)
keyword = '"covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working"'

### Merge csv's or SCRAPER new search

In [274]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])

  #get next data
  next_date = next_data(df)

  #get seach
  df_new = search(keyword, next_date)

  #get domain
  df_new = fetch_domain(df_new)

  #get hostname
  df_new = fetch_hostname(df_new)

  #get ipaddress
  df_new = fetch_ipaddress(df_new)

  #get geocoded ipaddress
  df_new = fetch_location(df_new)

  #get nlp contextualSpellCheck 
  df_new = fetch_nlp(df_new)

  #save df new
  save(df_new)

  #print new df
  df_new.head(2)
  
else:
  #get first date
  first_date = date(2021, 1, 1)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df = fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: use options instead of chrome_options


https://www.google.com/search?q="covid impact" OR "disaster" OR "resilience" AND "software engineers" OR "software development" OR "disaster informatics" AND "productivity" OR "work from home" OR "remote working" after:2021-04-21 and before:2021-04-23&sort=date:r:2021-04-21:2021-04-23&num=200
horizon = after:2021-04-21 and before:2021-04-23


counts : 100


,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
95,95,2021-04-20,NIC/IT/RFP/Collaborative Workspace Solution/RF...,productivity of workforce remotely. • Enable N...,https://nationalinsurance.nic.co.in/sites/defa...,https,nationalinsurance.nic.co.in,/sites/default/files/NIC%20RFP%20for%20Collabo...,NaN,NaN,nic.co.in,14.140.118.180,IN,India,West Bengal,Kolkata,88.345802,22.572701,4755.0,NaN,geolocation,{},14.140.118.180,NIC/IT/RFP/Collaborative Workspace Solution/RF...,productivity of workforce remotely. • Enable N...
96,96,2021-04-20,2020 Annual Impact Report | Upwork,20 apr. 2021 —,https://www.upwork.com/documents/2020-annual-i...,https,www.upwork.com,/documents/2020-annual-impact-report,NaN,NaN,upwork.com,104.18.89.237,US,United States,California,San Jose,-121.838226,37.330528,13335.0,NaN,geolocation,{},104.18.89.237,2020 Annual Impact Report | Upwork,20 apr. 2021 —
97,97,2021-04-20,Sustainable Agriculture in India 2021 - Green ...,Improving Women's Productivity and Incomes Thr...,https://www.greengrowthknowledge.org/research/...,https,www.greengrowthknowledge.org,/research/sustainable-agriculture-india-2021,NaN,NaN,greengrowthknowledge.org,3.83.28.40,US,United States,Virginia,Ashburn,-77.474197,39.043701,14618.0,NaN,geolocation,{},3.83.28.40,Sustainable Agriculture in India 2021 - Green ...,Improving Women's Productivity and Incomes Thr...
98,98,2021-04-20,Working In a Big-Tech Firm Turned Into a Night...,20 apr. 2021 —,https://betterprogramming.pub/working-in-a-big...,https,betterprogramming.pub,/working-in-a-big-tech-firm-turned-into-a-nigh...,NaN,NaN,betterprogramming.pub,162.159.153.4,BE,Belgium,Brussels Capital,Brussels,4.352490,50.846741,13335.0,NaN,geolocation,{},162.159.153.4,Working In a Big-Tech Firm Turned Into a Night...,20 apr. 2021 —
99,99,2021-04-20,Working In a Big-Tech Firm Turned Into a Night...,20 apr. 2021 —,https://betterprogramming.pub/working-in-a-big...,https,betterprogramming.pub,/working-in-a-big-tech-firm-turned-into-a-nigh...,NaN,NaN,betterprogramming.pub,162.159.153.4,BE,Belgium,Brussels Capital,Brussels,4.352490,50.846741,13335.0,NaN,geolocation,{},162.159.153.4,Working In a Big-Tech Firm Turned Into a Night...,20 apr. 2021 —


In [275]:
df_new.head()

,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,TimeGenerated,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2021-04-21,DevPrime - Unleash the power of software innov...,"... as shown in Figure 01, increasing producti...",https://devprime.tech/products/devprime,https,devprime.tech,/products/devprime,,,devprime.tech,52.188.43.174,US,United States,Virginia,Washington,-78.171967,38.730782,8075.0,None,geolocation,{},52.188.43.174,DevPrime - Unleash the power of software innov...,"... as shown in Figure 01, increasing producti..."
1,2021-04-21,Software Development | Dextra Technologies,"At Dextra Technologies we had the caution, agi...",https://dextratech.com/category/software-devel...,https,dextratech.com,/category/software-development/,,,dextratech.com,3.213.239.178,US,United States,Virginia,Ashburn,-77.474197,39.043701,14618.0,None,geolocation,{},3.213.239.178,Software Development | Dextra Technologies,"At Dextra Technologies we had the caution, agi..."
2,2021-04-21,In-House vs. Outsourced Software ... - Cosmic ...,22 apr. 2021 —,https://www.cosmicdevelopment.com/blog/in-hous...,https,www.cosmicdevelopment.com,/blog/in-house-vs-outsourced-software-developm...,,,cosmicdevelopment.com,138.197.145.73,CA,Canada,Ontario,Toronto,-79.512192,43.703171,14061.0,None,geolocation,{},138.197.145.73,In-House vs. Outsourced Software ... - Cosmic ...,22 apr. 2021 —
3,2021-04-21,"Rethinking the role of IT in a post-Brexit, po...",22 apr. 2021 —,https://www.velocity-it.com/knowledge-hub/reth...,https,www.velocity-it.com,/knowledge-hub/rethinking-the-role-of-it-in-a-...,,,velocity-it.com,217.160.0.37,DE,Germany,Baden-Württemberg,Karlsruhe,8.389790,49.013359,8560.0,None,geolocation,{},217.160.0.37,"Rethinking the role of IT in a post-Brexit, po...",22 apr. 2021 —
4,2021-04-21,Clutch 100 Top Software Development Companies ...,22 apr. 2021 —,https://www.scalosoft.com/blog/scalo-best-soft...,https,www.scalosoft.com,/blog/scalo-best-software-development-companies/,,,scalosoft.com,65.9.83.18,US,United States,Florida,Flagami,-80.311958,25.762859,16509.0,None,geolocation,{},65.9.83.18,Clutch 100 Top Software Development Companies ...,22 apr. 2021 —
